# mnist variational auto encoder 예제

https://github.com/lyeoni/pytorch-mnist-VAE/blob/master/pytorch-mnist-VAE.ipynb


In [3]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn # 인공 신경망 모델들 모아놓은 모듈
import torch.nn.functional as F #그중 자주 쓰이는것들을 F로
from torchvision import transforms, datasets
import cv2
from torchvision import transforms, datasets
import pandas as pd
import os
from glob import glob
import torchvision.models as models
import sys

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
#DEVICE = torch.device('cpu')
print('Using Pytorch version : ',torch.__version__,' Device : ',DEVICE)

Using Pytorch version :  1.10.2  Device :  cuda


In [4]:
bs = 100
# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data/', train=False, transform=transforms.ToTensor(), download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [11]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        # encoder part
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)
        # decoder part
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var) # e^(0.5*log_var)
        eps = torch.randn_like(std) # std사이즈와 같은 정규분포 랜덤 z값 생성. 
        
        
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return F.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var) 
        # 왜 encoder 결과가 mu와 log_var 인지?
        # 해당 이미지가 mu 에 위치하고, var만큼 퍼져있는 지역에 있다는 의미.
        # z라는 랜덤변수를 추출해서, decoder로.
        
        return self.decoder(z), mu, log_var

# build model
vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=2)
if torch.cuda.is_available():
    vae=vae.cuda()

In [12]:
vae

VAE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc31): Linear(in_features=256, out_features=2, bias=True)
  (fc32): Linear(in_features=256, out_features=2, bias=True)
  (fc4): Linear(in_features=2, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
)

In [19]:

optimizer = torch.optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses

def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum') #합이 같은지를 비교
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [20]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [22]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [23]:
for epoch in range(1, 51):
    train(epoch)
    test()

C:\Users\USER\anaconda3\envs\local_torch\lib\site-packages\torch\nn\functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [0/60000 (0%)]	Loss: 543.683867
Train Epoch: 1 [10000/60000 (17%)]	Loss: 195.131797
Train Epoch: 1 [20000/60000 (33%)]	Loss: 174.003652
Train Epoch: 1 [30000/60000 (50%)]	Loss: 170.172949
Train Epoch: 1 [40000/60000 (67%)]	Loss: 175.126191
Train Epoch: 1 [50000/60000 (83%)]	Loss: 169.975957
====> Epoch: 1 Average loss: 180.3331
====> Test set loss: 163.0279
Train Epoch: 2 [0/60000 (0%)]	Loss: 165.500586
Train Epoch: 2 [10000/60000 (17%)]	Loss: 151.593818
Train Epoch: 2 [20000/60000 (33%)]	Loss: 164.645508
Train Epoch: 2 [30000/60000 (50%)]	Loss: 157.385273
Train Epoch: 2 [40000/60000 (67%)]	Loss: 160.797227
Train Epoch: 2 [50000/60000 (83%)]	Loss: 156.149180
====> Epoch: 2 Average loss: 158.2077
====> Test set loss: 155.2007
Train Epoch: 3 [0/60000 (0%)]	Loss: 158.445703
Train Epoch: 3 [10000/60000 (17%)]	Loss: 158.338994
Train Epoch: 3 [20000/60000 (33%)]	Loss: 149.320742
Train Epoch: 3 [30000/60000 (50%)]	Loss: 154.243730
Train Epoch: 3 [40000/60000 (67%)]	Loss: 149.15

Train Epoch: 22 [50000/60000 (83%)]	Loss: 131.232832
====> Epoch: 22 Average loss: 138.0359
====> Test set loss: 139.7064
Train Epoch: 23 [0/60000 (0%)]	Loss: 148.735469
Train Epoch: 23 [10000/60000 (17%)]	Loss: 136.594014
Train Epoch: 23 [20000/60000 (33%)]	Loss: 138.548086
Train Epoch: 23 [30000/60000 (50%)]	Loss: 140.461250
Train Epoch: 23 [40000/60000 (67%)]	Loss: 139.803105
Train Epoch: 23 [50000/60000 (83%)]	Loss: 141.680684
====> Epoch: 23 Average loss: 138.0366
====> Test set loss: 140.0233
Train Epoch: 24 [0/60000 (0%)]	Loss: 139.768584
Train Epoch: 24 [10000/60000 (17%)]	Loss: 141.773242
Train Epoch: 24 [20000/60000 (33%)]	Loss: 132.628076
Train Epoch: 24 [30000/60000 (50%)]	Loss: 142.682412
Train Epoch: 24 [40000/60000 (67%)]	Loss: 136.555117
Train Epoch: 24 [50000/60000 (83%)]	Loss: 139.059297
====> Epoch: 24 Average loss: 137.5555
====> Test set loss: 139.2645
Train Epoch: 25 [0/60000 (0%)]	Loss: 138.031787
Train Epoch: 25 [10000/60000 (17%)]	Loss: 131.931973
Train Epoch: 

Train Epoch: 44 [20000/60000 (33%)]	Loss: 134.360811
Train Epoch: 44 [30000/60000 (50%)]	Loss: 137.609990
Train Epoch: 44 [40000/60000 (67%)]	Loss: 137.806387
Train Epoch: 44 [50000/60000 (83%)]	Loss: 134.041270
====> Epoch: 44 Average loss: 135.2151
====> Test set loss: 138.2302
Train Epoch: 45 [0/60000 (0%)]	Loss: 146.704863
Train Epoch: 45 [10000/60000 (17%)]	Loss: 135.331992
Train Epoch: 45 [20000/60000 (33%)]	Loss: 134.255957
Train Epoch: 45 [30000/60000 (50%)]	Loss: 134.245098
Train Epoch: 45 [40000/60000 (67%)]	Loss: 136.532969
Train Epoch: 45 [50000/60000 (83%)]	Loss: 133.092666
====> Epoch: 45 Average loss: 135.1306
====> Test set loss: 138.4556
Train Epoch: 46 [0/60000 (0%)]	Loss: 143.874004
Train Epoch: 46 [10000/60000 (17%)]	Loss: 139.469844
Train Epoch: 46 [20000/60000 (33%)]	Loss: 136.076260
Train Epoch: 46 [30000/60000 (50%)]	Loss: 125.654609
Train Epoch: 46 [40000/60000 (67%)]	Loss: 132.893281
Train Epoch: 46 [50000/60000 (83%)]	Loss: 140.751387
====> Epoch: 46 Average 

In [28]:
from torchvision.utils import save_image

In [31]:
with torch.no_grad():
    z = torch.randn(64, 2).cuda()
    sample = vae.decoder(z).cuda()
    
    save_image(sample.view(64, 1, 28, 28), './samples/sample_' + '.png')

In [30]:
sample.size()

torch.Size([64, 784])